# Car Rental 2

## Objective and Prerequisites

In this example, which is an extension of Vehicle Rental Optimization I, you’ll see how mathematical optimization can be used to figure out in which locations a car rental company should expand repair capacity. We’ll guide you through the process of creating a mixed integer linear programming model of this problem using the Gurobi Python API and then show you how you can find an optimal solution with the Gurobi Optimizer.

This model is example 26 from the fifth edition of Model Building in Mathematical Programming by H. Paul Williams on pages 287 and 342-343.

This example is at the intermediate level, where we assume that you know Python and the Gurobi Python API and that you have some knowledge of building mathematical optimization models.

**Download the Repository** <br /> 
You can download the repository containing this and other examples by clicking [here](https://github.com/Gurobi/modeling-examples/archive/master.zip). 

---
## Problem Description

A small car rental company
(which rents only one type of car) 
has depots in Glasgow, Manchester, Birmingham and Plymouth. There is an estimated demand for each day of the week except Sunday (when the company is closed). These estimates are given in the following table. It is not necessary to meet all demand.

![weeklyDemand](weeklyDemand.PNG)


Cars can be rented for one, two or three days and returned to either the
depot from which they were rented or another depot at the start of the next morning. For
example, a 2-day rental on Thursday means that the car has to be returned on
Saturday morning; a 3-day rental on Friday means that the car has to be returned
on Tuesday morning. A 1-day rental on Saturday means that the car has to be
returned on Monday morning, and a 2-day rental on Tuesday morning.

The rental period is independent of the origin and destination. From historical
data, the company knows the distribution of rental periods: 55% of cars are rented
for one day, 20% for two days and 25% for three days. The current estimates
of percentages of cars rented from each depot and returned to a given depot
(independent of the day) are given in the following table.

![FromToPct](FromToPct.PNG)

The company's marginal cost
of renting out a car (‘wear and tear’, administration, etc.) is estimated as follows:

| Days rented | Marginal cost |
| --- | --- |
| 1-day | $\$ 20$ |
| 2-day | $\$ 25$ |
| 3-day | $\$ 30$ |

The ‘opportunity cost’ (interest on capital, storage, servicing, etc.) of owning
a car is $\$ 15$ per week.

It is possible to transfer undamaged cars from one depot to another depot,
irrespective of distance. Cars cannot be rented out during the day in which they
are transferred. The costs in USD, per car, of transfer are given in the following table.

![FromToCst](FromToCst.PNG)

Ten percent of cars returned by customers are damaged. When this happens,
the customer is charged an excess of $\$ 100$  (irrespective of the amount of damage
that the company completely covers by its insurance). In addition, the car has
to be transferred to a repair depot, where it will be repaired the following day.
The cost of transferring a damaged car is the same as transferring an undamaged
one (except when the repair depot is the current depot, 
in which case the cost would be $\$0$). 
The transfer of a damaged car takes a day, unless it is already at a repair depot.
Having arrived at a repair depot, all types of repair (or replacement) take a day.
Only two of the depots have repair capacity. The (cars/day) capacity at each repair depot is as follows:

| Repair depot | Capacity |
| --- | --- |
| Manchester | 12 |
| Birmingham | 20 |

Having been repaired, the car is available for rent at the depot the next day
or may be transferred to another depot (taking a day). Thus, a car that is returned
damaged on a Wednesday morning is transferred to a repair depot (if not the
current depot) on Wednesday, repaired on Thursday and is available for rent
at the repair depot on Friday morning.
The rental price depends on the number of days for which the car is rented
and whether it is returned to the same depot or not. The prices (in USD) are given in the following table.

![RentalPrice](RentalPrice.PNG)

We assume the following at the beginning of each day:
1. Customers return cars that are due that day.
2. Damaged cars are sent to the repair depot.
3. Cars that were transferred from other depots arrive.
4. Transfers are sent out.
5. Cars are rented out.
6. If it is a repair depot, then the repaired cars are available for rental.

The car rental company wants to determine in which locations it should expand  repair capacity. The weekly fixed costs, given below, include interest payments on the necessary
loans for expansion. The options are as follows:

1. Expand repair capacity at Birmingham by 5 cars per day at a fixed cost
per week of $\$18,000$.
2. Further expand repair capacity at Birmingham by 5 cars per day at a fixed
cost per week of $\$8,000$.
3. Expand repair capacity at Manchester by 5 cars per day at a fixed cost per
week of $\$20,000$.
4. Further expand repair capacity at Manchester by 5 cars per day at a fixed
cost per week of $\$5,000$.
5. Create repair capacity at Plymouth of 5 cars per day at a fixed cost per
week of $\$19,000$.

If any of these options is chosen, it must be carried out in its entirety; that
is, there can be no partial expansion. Also, a further expansion at a depot can be
carried out only if the first expansion is also carried out -so, for example, option
(2) at Birmingham cannot be chosen unless option (1) is also chosen. If option (2)
is chosen, thereby also choosing option (1), these count as two options. Similar
stipulations apply regarding the expansions at Manchester. At most three of the
options can be carried out.

The goal is to determine in which locations to expand  repair capacity, the numbers of cars the car rental company should own, and where they should  be located at the start of each week day in order to maximize weekly profit. The company wants a ‘steady state’ solution in which the same expected number of cars will be located at the same depot on the same day of subsequent weeks.

---
## Model Formulation

$d,d2 \in \text{Depots}=\{\text{Glasgow}, \text{Manchester}, \text{Birmingham},  \text{Plymouth}\}$

$\text{NRD}=\{\text{Glasgow}\}$: Depots without repair capacity.

$\text{RD}=\{\text{Manchester}, \text{Birmingham}, \text{Plymouth}\}$: Depots with repair capacity.

$t \in \text{Days}=\{\text{Monday},\text{Tuesday},\text{Wednesday},\text{Thursday},\text{Friday},\text{Saturday}\}$

$r \in \text{RentDays}=\{1,2,3\}$: Number of days rented.

### Parameters

$\text{demand}_{d,t} \in \mathbb{R}^+$: Estimated rental demand at depot $d$ on day $t$.

$\text{pctDepot}_{d,d2} \in \mathbb{R}^+$: Proportion of cars rented at depot $d$ to be returned to depot $d2$.

$\text{cstTransfer}_{d,d2} \in \mathbb{R}^+$: Transfer cost of a car from depot $d$ to depot $d2$.

$\text{pctRent}_{r} \in \mathbb{R}^+$: Proportion of cars rented for $r$ days.

$\text{capRepair}_{d} \in \mathbb{R}^+$: Repair capacity of depot $d$.

$\text{cstSameDepot}_{r} \in \mathbb{R}^+$: Rental cost for $r$ days with return to same depot.

$\text{cstOtherDepot}_{r} \in \mathbb{R}^+$: Rental cost for $r$ days with return to other depot.

$\text{marginalCost}_{r} \in \mathbb{R}^+$: Marginal cost to company of  $r$ days rental of a car.

$\text{pctUndamaged } \in [0,1]$: Percent of cars returned by customers that are undamaged.

$\text{pctDamaged  } \in [0,1]$: Percent of cars returned by customers that are damaged.

$\text{cstOwn} \in \mathbb{R}^+$: Cost of owning a car.

$\text{damagedFee} = 10$: Damaged car fee. Ten percent of the cars are damaged and the fee for a damaged car is $\$100$.

$\text{cstExpCapB1} \in \mathbb{R}^+$: Fixed cost per week of expanding capacity at Birmingham.

$\text{cstExpCapB2} \in \mathbb{R}^+$: Fixed cost per week of further expanding capacity at Birmingham.

$\text{cstExpCapM1} \in \mathbb{R}^+$: Fixed cost per week of expanding capacity at Manchester.

$\text{cstExpCapM2} \in \mathbb{R}^+$: Fixed cost per week of further expanding capacity at Manchester.

$\text{cstExpCapP} \in \mathbb{R}^+$: Fixed cost per week of expanding capacity at Plymouth.

### Decision Variables

$\text{xOwned} \in \mathbb{R}^+$: Total number of cars owned.

$\text{xUndamaged}_{d,t} \in \mathbb{R}^+$: Number of undamaged cars available at depot $d$ at the beginning of day $t$.

$\text{xDamaged}_{d,t} \in \mathbb{R}^+$: Number of damaged cars available at depot $d$ at the beginning of day $t$.

$\text{xRented}_{d,t} \in \mathbb{R}^+$: number of cars rented  out from depot $d$ at the beginning of day $t$.

$\text{xUDleft}_{d,t} \in \mathbb{R}^+$: Number of undamaged cars available at depot $d$ at the beginning of day $t$.

$\text{xDleft}_{d,t} \in \mathbb{R}^+$: Number of damaged cars left in depo $d$ at the end of day  $t$.

$\text{xUDtransfer}_{d,d2,t} \in \mathbb{R}^+$: Number of undamaged cars  at depot $d$ at the beginning of day $t$, to be transferred to depot $d2$. 

$\text{xDtransfer}_{d,d2,t} \in \mathbb{R}^+$: Number of damaged cars at depot $d$ at the beginning of day $t$, to be transferred to depot $d2$. 

$\text{xRepaired}_{d,t} \in \mathbb{R}^+$: Number of damaged cars to be repaired at depot $d$ during day $t$.

$\text{yExpandCapB1} \in \{0,1 \}$: This binary variable is equal to 1, if the Birmingham repair capacity is expanded by 5 cars per day.

$\text{yExpandCapB2} \in \{0,1 \}$: This binary variable is equal to 1, if the Birmingham repair capacity is expanded by a further 5 cars per day.

$\text{yExpandCapM1} \in \{0,1 \}$: This binary variable is equal to 1, if the Manchester repair capacity is expanded by 5 cars per day.

$\text{yExpandCapM2} \in \{0,1 \}$: This binary variable is equal to 1, if the Manchester repair capacity is expanded by a further 5 cars per day.

$\text{yExpandCapP} \in \{0,1 \}$: This binary variable is equal to 1, if the Plymouth repair capacity is expanded by 5 cars per day.

### Objective function
The objective is to maximize profit.

\begin{equation}
\sum_{d \in \text{Depots}}
\sum_{t \in \text{Days}}
\sum_{r \in \text{RentDays}} 
\text{pctDepot}_{d,d}*\text{pctRent}_{r}*(\text{cstSameDepot}_{r} - \text{marginalCost}_{r} + \text{damagedFee})*\text{xRented}_{d,t}
\end{equation}

\begin{equation}
+ \sum_{d \in \text{Depots}} \sum_{d2 \in \text{Depots}}
\sum_{t \in \text{Days}}
\sum_{r \in \text{RentDays}} 
\text{pctDepot}_{d,d2}*\text{pctRent}_{r}*(\text{cstOtherDepot}_{r} - \text{marginalCost}_{r} + \text{damagedFee})*\text{xRented}_{d,t}
\end{equation}

\begin{equation}
- \sum_{d \in \text{Depots}} \sum_{d2 \in \text{Depots}}
\sum_{t \in \text{Days}} \text{cstTransfer}_{d,d2}*(\text{xUDtransfer}_{d,d2,t} + \text{xDtransfer}_{d,d2,t} )
- \text{cstOwn}*\text{xOwned}
\end{equation}

\begin{equation}
-(\text{cstExpCapB1}*\text{yExpandCapB1} + \text{cstExpCapB2}*\text{yExpandCapB2} + \text{cstExpCapM1}*\text{yExpandCapM1} 
\end{equation}

\begin{equation}
+ \text{cstExpCapM2}*\text{yExpandCapM2} + \text{cstExpCapP}*\text{yExpandCapP} )
\end{equation}

### Constraints

**Undamaged cars at a non-repair depot** <br />
Number of undamaged cars available at a non-repair depot $d$ at the beginning of day $t$.

\begin{equation}
\sum_{d2 \in \text{Depots}} 
\sum_{r \in \text{RentDays}} \text{pctUndamaged}*\text{pctDepot}_{d2,d}*\text{pctRent}_{r}*\text{xRented}_{d2,(t-r)mod(6)}
\end{equation}

\begin{equation}
+ \sum_{d2 \in \text{Depots}} \text{xUDtransfer}_{d2,d,(t-1)mod(6)} + \text{xUDleft}_{d,(t-1)mod(6)} = \text{xUndamaged}_{d,t} 
\quad \forall d \in NRD, t \in Days
\end{equation}

Demand of undamaged cars at the non-repair depot $d$ during day $t$.

\begin{equation}
\text{xUndamaged}_{d,t} = \text{xRented}_{d,t} + 
\sum_{d2 \in \text{Depots}} \text{xUDtransfer}_{d,d2,t} + \text{xUDleft}_{d,t}
\quad \forall d \in NRD, t \in Days
\end{equation}

**Undamaged cars at a repair depot** <br />
Number of undamaged cars available at a repair depot $d$ at the beginning of day $t$.

\begin{equation}
\sum_{d2 \in \text{Depots}} 
\sum_{r \in \text{RentDays}} \text{pctUndamaged}*\text{pctDepot}_{d2,d}*\text{pctRent}_{r}*\text{xRented}_{d2,(t-r)mod(6)}
\end{equation}

\begin{equation}
+ \sum_{d2 \in \text{Depots}} \text{xUDtransfer}_{d2,d,(t-1)mod(6)} 
+ \text{xRepaired}_{d,(t-1)mod(6)}  + \text{xUDleft}_{d,(t-1)mod(6)} = \text{xUndamaged}_{d,t} 
\quad \forall d \in NRD, t \in Days
\end{equation}

Demand of undamaged cars at the repair depot $d$ during day $t$.

\begin{equation}
\text{xUndamaged}_{d,t} = \text{xRented}_{d,t} + 
\sum_{d2 \in \text{Depots}} \text{xUDtransfer}_{d,d2,t} + \text{xUDleft}_{d,t}
\quad \forall d \in NRD, t \in Days
\end{equation}

**Damaged cars at a non-repair depot** <br />
Number of damaged cars available at a non-repair depot $d$ at the beginning of day $t$.

\begin{equation}
\sum_{d2 \in \text{Depots}} 
\sum_{r \in \text{RentDays}} \text{pctDamaged}*\text{pctDepot}_{d2,d}*\text{pctRent}_{r}*\text{xRented}_{d2,(t-r)mod(6)}
\end{equation}

\begin{equation}
+ \text{xDleft}_{d,(t-1)mod(6)} = \text{xDamaged}_{d,t} \quad \forall d \in NRD, t \in Days
\end{equation}

Demand of undamaged cars at the non-repair depot $d$ during day $t$.

\begin{equation}
\text{xDamaged}_{d,t} = 
\sum_{d2 \in \text{Depots} \cap RD} \text{xDtransfer}_{d,d2,t} + \text{xDleft}_{d,t}
\quad \forall d \in NRD, t \in Days
\end{equation}

**Damaged cars at a repair depot** <br />
Number of damaged cars available at a repair depot $d$ at the beginning of day $t$.

\begin{equation}
\sum_{d2 \in \text{Depots}} 
\sum_{r \in \text{RentDays}} \text{pctDamaged}*\text{pctDepot}_{d2,d}*\text{pctRent}_{r}*\text{xRented}_{d2,(t-r)mod(6)}
\end{equation}

\begin{equation}
+ \sum_{d2 \in \text{Depots}} \text{xDtransfer}_{d2,d,(t-1)mod(6)} 
+ \text{xDleft}_{d,(t-1)mod(6)} = \text{xdamaged}_{d,t} 
\quad \forall d \in RD, t \in Days
\end{equation}

Demand of undamaged cars at the non-repair depot $d$ during day $t$.

\begin{equation}
\text{xDamaged}_{d,t} = \text{xRepaired}_{d,t} +
\sum_{d2 \in \text{Depots} \cap RD} \text{xDtransfer}_{d,d2,t} + \text{xDleft}_{d,t}
\quad \forall d \in RD, t \in Days
\end{equation}

**Repair Capacity** <br />
Repair capacity of depot $d$ for each day $t$, including expansion capacity options. 

\begin{equation}
\text{xRepaired}_{Birmingham,t} \leq \text{capRepair}_{Birmingham}
+ 5*\text{yExpandCapB1} + 5*\text{yExpandCapB2}
\quad \forall t \in Days
\end{equation}

\begin{equation}
\text{xRepaired}_{Manchester,t} \leq \text{capRepair}_{Manchester}
+ 5*\text{yExpandCapM1} + 5*\text{yExpandCapM2}
\quad \forall t \in Days
\end{equation}

\begin{equation}
\text{xRepaired}_{Plymouth,t} \leq \text{capRepair}_{Plymouth}
+ 5*\text{yExpandCapP}
\quad \forall t \in Days
\end{equation}

**Depot Demand** <br />
Demand at depot $d$ for each day $t$. 

\begin{equation}
\text{xRented}_{d,t} \leq \text{demand}_{d,t}
\quad \forall d \in Depots, t \in Days
\end{equation}

**Number of cars** <br />
Total number of cars owned equals number of cars rented out from all depots on Monday for 3 days, plus those on Tuesday for 2 or 3 days, plus all damaged and undamaged cars in depots at the beginning of Wednesday. 
Rationale: Let’s choose a day (Wednesday), count the cars undamaged and damaged that were returned to the depots and that are available on Wednesday morning. Let’s count the cars that have been rented and have not been returned: Cars rented on Monday for 3 days, and cars rented on Tuesday for 2 or 3 days.

\begin{equation}
\sum_{d \in \text{Depots}} (0.25*\text{xRented}_{d,0} + 0.45*\text{xRented}_{d,1} + \text{xUndamaged}_{d,2}  + \text{xdamaged}_{d,2} ) = \text{xOwned}
\end{equation}

**Capacity expansion constraints** <br />
Additional expansion at a depot can be carried out only if the first expansion is also carried out.

\begin{equation}
\text{yExpandCapB1} \geq \text{yExpandCapB2}
\end{equation}

\begin{equation}
\text{yExpandCapM1} \geq \text{yExpandCapM2}
\end{equation}

**Maximum number of expansions** <br />
At most three of the options can be carried out.

\begin{equation}
\text{yExpandCapB1} + \text{yExpandCapB2} +
\text{yExpandCapM1} + 
\end{equation}

\begin{equation}
\text{yExpandCapM2} +
\text{yExpandCapP} \leq 3
\end{equation}


---
## Python Implementation

We import the Gurobi Python Module and other Python libraries.

In [ ]:
%pip install gurobipy

In [1]:
import pandas as pd
from itertools import product

import gurobipy as gp
from gurobipy import GRB

# tested with Python 3.11 & Gurobi 11.0

## Input data

We define all the input data for the model.

In [2]:
# list of depots and working days of a week

depots = ['Glasgow','Manchester','Birmingham','Plymouth']
NRD = ['Glasgow'] # Non-repair depot
RD =['Manchester','Birmingham','Plymouth'] # Repair depot

days = [0,1,2,3,4,5] # Monday = 0, Tuesday = 1, ...  Saturday = 5
rentDays = [1,2,3]

d2w, demand = gp.multidict({
    ('Glasgow',0): 100,
    ('Glasgow',1): 150,
    ('Glasgow',2): 135,
    ('Glasgow',3): 83,
    ('Glasgow',4): 120,
    ('Glasgow',5): 230,
    ('Manchester',0): 250,
    ('Manchester',1): 143,
    ('Manchester',2): 80,
    ('Manchester',3): 225,
    ('Manchester',4): 210,
    ('Manchester',5): 98,
    ('Birmingham',0): 95,
    ('Birmingham',1): 195,
    ('Birmingham',2): 242,
    ('Birmingham',3): 111,
    ('Birmingham',4): 70,
    ('Birmingham',5): 124,
    ('Plymouth',0): 160,
    ('Plymouth',1): 99,
    ('Plymouth',2): 55,
    ('Plymouth',3): 96,
    ('Plymouth',4): 115,
    ('Plymouth',5): 80
})

#repairCap
depots, capacity = gp.multidict({
    ('Glasgow'): 0,
    ('Manchester'): 12,
    ('Birmingham'): 20,
    ('Plymouth'): 0
})

# Create a dictionary to capture 
# pctRent: percentage of cars rented for r days 
# cstMarginal: marginal cost for renting a car for r days
# prcSameD: price of renting a car r days and returning to same depot
# prcOtherD: price of renting a car r days and returning to another depot
rentDays, pctRent, costMarginal, priceSameD, priceOtherD = gp.multidict({
    (1): [0.55,20,50,70],
    (2): [0.20,25,70,100],
    (3): [0.25,30,120,150]
})

# Cost of owing a car per week.
cstOwn = 15

# damaged car fee
damagedFee = 10

# Create a dictionary to capture the proportion of cars rented at depot d to be returned to depot d2 
d2d, pctFromToD = gp.multidict({
    ('Glasgow','Glasgow'): 0.6,
    ('Glasgow','Manchester'): 0.2,
    ('Glasgow','Birmingham'): 0.1,
    ('Glasgow','Plymouth'): 0.1,
    ('Manchester','Glasgow'): 0.15,
    ('Manchester','Manchester'): 0.55,
    ('Manchester','Birmingham'): 0.25,
    ('Manchester','Plymouth'): 0.05,
    ('Birmingham','Glasgow'): 0.15,
    ('Birmingham','Manchester'): 0.2,
    ('Birmingham','Birmingham'): 0.54,
    ('Birmingham','Plymouth'): 0.11,
    ('Plymouth','Glasgow'): 0.08,
    ('Plymouth','Manchester'): 0.12,
    ('Plymouth','Birmingham'): 0.27,
    ('Plymouth','Plymouth'): 0.53
})

# Create a dictionary to capture the transfer costs  of cars
d2d, cstFromToD = gp.multidict({
    ('Glasgow','Glasgow'): 0.001,
    ('Glasgow','Manchester'): 20,
    ('Glasgow','Birmingham'): 30,
    ('Glasgow','Plymouth'): 50,
    ('Manchester','Glasgow'): 20,
    ('Manchester','Manchester'): 0.001,
    ('Manchester','Birmingham'): 15,
    ('Manchester','Plymouth'): 35,
    ('Birmingham','Glasgow'): 30,
    ('Birmingham','Manchester'): 15,
    ('Birmingham','Birmingham'): 0.001,
    ('Birmingham','Plymouth'): 25,
    ('Plymouth','Glasgow'): 50,
    ('Plymouth','Manchester'): 35,
    ('Plymouth','Birmingham'): 25,
    ('Plymouth','Plymouth'): 0.001
})

# Proportion of undamaged and damaged cars returned
pctUndamaged = 0.9
pctDamaged = 0.1

# Expansion of capacity binary costs
cstExpCapB1 = 18000
cstExpCapB2 = 8000
cstExpCapM1 = 20000
cstExpCapM2 = 5000
cstExpCapP = 19000


### Preprocessing
We prepare the data structures to build the mixed integer linear programming model.

In [3]:
# Build a list of tuples (depot, depot2) such that d != d2
list_d2notd = []

for d,d2 in d2d:
    if (d != d2):
        tp = d,d2
        list_d2notd.append(tp)

d2notd = gp.tuplelist(list_d2notd)

# Build a list of tuples (depot, depot2, day)
list_dd2t = []

for d,d2 in d2notd:
    for t in days:
        tp = d,d2,t 
        list_dd2t.append(tp)
                    
dd2t = gp.tuplelist(list_dd2t)

# Build a list of tuples (depot, rent_day)
list_dr = []

for d in depots:
    for r in rentDays:
        tp = d,r
        list_dr.append(tp)
        
dr = gp.tuplelist(list_dr) 

# Build a list of tuples (depot, day, rent_days )
list_dtr = []

for d in depots:
    for t in days:
            for r in rentDays:
                tp = d,t,r
                list_dtr.append(tp)
                
dtr = gp.tuplelist(list_dtr) 

# Build a list of tuples (depot, depot2, day, rent_days)
list_dd2tr = []

for d,d2 in d2notd:
    for t in days:
        for r in rentDays:
            tp = d,d2,t,r
            list_dd2tr.append(tp)
                    
                    
dd2tr = gp.tuplelist(list_dd2tr)

## Model Deployment
We create a model and the variables. The main decision variables are to choose the depots were repair capacity is 
worthwhile to expand, the number of cars to own
and where should they be located at the start of each day of a week to maximize weekly profits.

In [4]:
model = gp.Model('RentalCar1')

# Number of cars owned
n = model.addVar(name="cars")

# Number of undamaged cars
nu = model.addVars(d2w, name="UDcars")

# Number of damaged cars
nd = model.addVars(d2w, name="Dcars")

# Number of cars hired (rented) cannot exceed their demand
tr = model.addVars(d2w, ub=demand, name="Hcars")
#for d,t in d2w:
    #tr[d,t].lb = 1

# End inventory of undamaged cars
eu = model.addVars(d2w, name="EUDcars")

# End inventory of damaged cars
ed = model.addVars(d2w, name="EDcars")

# Number of undamaged cars transferred
tu = model.addVars(dd2t, name="TUDcars")

# Number of damaged cars transferred
td = model.addVars(dd2t, name="TDcars")

# Number of damaged cars repaired
rp = model.addVars(d2w, name="RPcars")

# Number of damaged cars repaired cannot exceed depot capacity
for d,t in d2w:
    if d == 'Glasgow':
        rp[d,t].ub = capacity[d] #repair capacity of Glasgow is zero
        
# Expansion of capacity binary variables
zB1 = model.addVar(vtype=GRB.BINARY, name='expCapB1') # Expansion at Birmingham

zB2 = model.addVar(vtype=GRB.BINARY, name='expCapB2') # Extra expansion at Birmingham

zM1 = model.addVar(vtype=GRB.BINARY, name='expCapM1') # Expansion at Manchester

zM2 = model.addVar(vtype=GRB.BINARY, name='expCapM2') # Extra expansion at Manchester

zP = model.addVar(vtype=GRB.BINARY, name='expCapP') # Expansion at Plymouth

Using license file c:\gurobi\gurobi.lic


### Constraints
The number of undamaged cars available at a non-repair depot d at the beginning of  day t should be equal to the demand of undamaged cars at the non-repair depot d during day t.

In [5]:
# Undamaged cars into a non-repair depot constraints (left hand side of balance equation -availability)

UDcarsNRD_L = model.addConstrs((gp.quicksum(pctUndamaged*pctFromToD[d2,d]*pctRent[r]*tr[d2,(t-r)%6 ] for d2,r in dr ) 
                              + gp.quicksum(tu.select('*',d,(t-1)%6)  ) 
                              + eu[d,(t-1)%6 ] == nu[d,t] for d in NRD for t in days ), 
                             name="UDcarsNRD_L")

# Undamaged cars out of a non-repair depot constraints (right hand side of balance equation -requirements)

UDcarsNRD_R = model.addConstrs((tr[d,t] 
                                + gp.quicksum(tu.select(d,'*',t )) 
                                + eu[d,t] == nu[d,t] for d in NRD for t in days ), name='UDcarsNRD_R' )

The number of undamaged cars available at a repair depot d at the beginning of  day t should be equal to the demand of undamaged cars at the repair depot d during day t.

In [6]:
# Undamaged cars into a repair depot constraints (left hand side of balance equation -availability)

UDcarsRD_L = model.addConstrs((gp.quicksum(pctUndamaged*pctFromToD[d2,d]*pctRent[r]*tr[d2,(t-r)%6 ] for d2,r in dr ) 
                              + gp.quicksum(tu.select('*',d,(t-1)%6)  ) + rp[d, (t-1)%6 ]
                              + eu[d,(t-1)%6 ] == nu[d,t] for d in RD for t in days ), 
                             name="UDcarsRD_L")

# Undamaged cars out of a repair depot constraints (right hand side of balance equation -requirements)

UDcarsRD_R = model.addConstrs((tr[d,t] 
                                + gp.quicksum(tu.select(d,'*',t ) ) 
                                + eu[d,t] == nu[d,t] for d in RD for t in days ), name='UDcarsRD_R' )

The number of damaged cars available at a non-repair depot d at the beginning of  day t should be equal to the demand of damaged cars at the non-repair depot d during day t.

In [7]:
# Damaged cars into a non-repair depot constraints (left hand side of balance equation -availability)

DcarsNRD_L = model.addConstrs((gp.quicksum(pctDamaged*pctFromToD[d2,d]*pctRent[r]*tr[d2,(t-r)%6 ] for d2,r in dr ) 
                              + ed[d,(t-1)%6 ] == nd[d,t] for d in NRD for t in days ), 
                             name="DcarsNRD_L")

# Damaged cars out of a non-repair depot constraints (right hand side of balance equation -requirements)

DcarsNRD_R = model.addConstrs(( gp.quicksum(td[d,d2,t] for d2 in RD ) 
                                + ed[d,t] == nd[d,t] for d in NRD for t in days ), name='DcarsNRD_R' )

The number of damaged cars available at a repair depot d at the beginning of  day t should be equal to the demand of damaged cars at the repair depot d during day t.

In [8]:
# Damaged cars into a repair depot constraints (left hand side of balance equation -availability)

DcarsRD_L = model.addConstrs((gp.quicksum(pctDamaged*pctFromToD[d2,d]*pctRent[r]*tr[d2,(t-r)%6 ] for d2,r in dr )
                              + gp.quicksum(td[d2,d,(t-1)%6 ] for d2, dd in d2notd if (dd == d)) 
                              + ed[d,(t-1)%6 ] == nd[d,t] for d in RD for t in days ), 
                             name="DcarsRD_L")

# Damaged cars out of a repair depot constraints (right hand side of balance equation -requirements)

DcarsND_R = model.addConstrs((rp[d,t] + gp.quicksum(td[d,d2,t ] for d2 in NRD ) 
                                + ed[d,t] == nd[d,t] for d in RD for t in days ), name='DcarsND_R' )

Total number of cars equals the number of cars rented out from all depots on Monday for 3 days, plus those on Tuesday for 2 or 3 days, plus all damaged and undamaged cars in depots at the beginning of Wednesday.

In [9]:
# Total number of cars owned constraint
# Note: 25% of cars are rented for 3 days, and 20% + 25% = 45% of the cars are rented for 2-days or 3-days

carsConstr = model.addConstr((gp.quicksum(0.25*tr[d,0] + 0.45*tr[d,1] + nu[d,2] + nd[d,2] for d in depots ) 
                              == n ),name='carsConstr')

Repair capacity expansion constraints.

In [10]:
# Depot capacity constraints

RepairCapB = model.addConstrs((rp['Birmingham',t] <= capacity['Birmingham'] 
                               + 5*(zB1 + zB2) for t in days ), name='RepairCapB')

RepairCapM = model.addConstrs((rp['Manchester',t] <= capacity['Manchester'] 
                               + 5*(zM1 + zM2) for t in days ), name='RepairCapM')

RepairCapP = model.addConstrs((rp['Plymouth',t] <= capacity['Plymouth'] 
                               + 5*zP for t in days ), name='RepairCapP')

# Additional expansion constraints

expandB1B2 = model.addConstr((zB1 >= zB2 ), name='expandB1B2')

expandM1M2 = model.addConstr((zM1 >= zM2 ), name='expandM1M2')

# At most three expansions are possible

atMost3 = model.addConstr((zB1 + zB2 + zM1 + zM2 + zP <= 3), name='atMost3')

The objective function is to maximize profit.

In [11]:
# Maximize profit objective function.

model.setObjective((
    gp.quicksum(pctFromToD[d,d]*pctRent[r]*(priceSameD[r] - costMarginal[r] + damagedFee)*tr[d,t] for d,t,r in dtr )
    + gp.quicksum(pctFromToD[d,d2]*pctRent[r]*(priceOtherD[r]-costMarginal[r]+damagedFee)*tr[d,t] for d,d2,t,r in dd2tr)
    - gp.quicksum(cstFromToD[d,d2]*tu[d,d2,t] for d,d2,t in dd2t) 
    - gp.quicksum(cstFromToD[d,d2]*td[d,d2,t] for d,d2,t in dd2t) 
    -(cstExpCapB1*zB1 + cstExpCapB2*zB2 + cstExpCapM1*zM1 + cstExpCapM2*zM2 + cstExpCapP*zP) - cstOwn*n ), GRB.MAXIMIZE)

In [12]:
# Verify model formulation

model.write('CarRental2.lp')

# Run optimization engine

model.optimize()

Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 118 rows, 294 columns and 1136 nonzeros
Model fingerprint: 0xe441c90b
Variable types: 289 continuous, 5 integer (5 binary)
Coefficient statistics:
  Matrix range     [1e-03, 5e+00]
  Objective range  [2e+01, 2e+04]
  Bounds range     [1e+00, 3e+02]
  RHS range        [3e+00, 2e+01]
Found heuristic solution: objective -0.0000000
Presolve removed 49 rows and 91 columns
Presolve time: 0.00s
Presolved: 69 rows, 203 columns, 987 nonzeros
Variable types: 198 continuous, 5 integer (5 binary)

Root relaxation: objective 1.357349e+05, 58 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 135734.925    0    2   -0.00000 135734.925      -     -    0s
H    0     0                    130734.92534 135734.925  3.8

---
## Analysis

In [13]:
# Output report

# Total number of cars owned
print(f"The optimal number of cars to be owned is: {round(n.x)}")

# Optimal profit
print(f"The optimal profit is: {'${:,.2f}'.format(round(model.objVal,2))}")


The optimal number of cars to be owned is: 983
The optimal profit is: $132,341.47


In [14]:
# Repair capacity expansion plan

if zB1.x > 0.5:
    print(f"The depot at Birmingham expanded its capacity to 5 more cars per day")
    if zB2.x > 0.5:
        print(f"The depot at Birmingham further expanded 5 more cars per day of additional capacity")

if zM1.x > 0.5:
    print(f"The depot at Manchester expanded its capacity to 5 more cars per day")
    if zM2.x > 0.5:
        print(f"The depot at Manchester further expanded 5 more cars per day of additional capacity")

if zP.x > 0.5:
    print(f"The depot at Plymouth expanded its capacity to 5 more cars per day")

The depot at Manchester expanded its capacity to 5 more cars per day
The depot at Manchester further expanded 5 more cars per day of additional capacity
The depot at Plymouth expanded its capacity to 5 more cars per day


In [15]:
# Create a list to translate the number label of each day to the actual name of the day
dayname = ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday']

# Number of undamaged cars in depot at the beginning of each day.
print("\n\n_________________________________________________________________________________")
print(f"Estimated number of undamaged cars in depot at the beginning of each day: ")
print("_________________________________________________________________________________")

undamaged_cars = pd.DataFrame(
    {
        "Day": [dayname[t] for t in days],
        "Glasgow": [round(nu['Glasgow',t].x) for t in days],
        "Manchester": [round(nu['Manchester',t].x) for t in days],
        "Birmingham": [round(nu['Birmingham',t].x) for t in days],
        "Plymouth": [round(nu['Plymouth',t].x) for t in days],
    }
)
undamaged_cars.index=[''] * len(undamaged_cars)
undamaged_cars



_________________________________________________________________________________
Estimated number of undamaged cars in depot at the beginning of each day: 
_________________________________________________________________________________


,Day,Glasgow,Manchester,Birmingham,Plymouth
,Monday,95,227,242,66
,Tuesday,99,162,282,65
,Wednesday,100,168,242,67
,Thursday,101,233,168,80
,Friday,117,226,162,71
,Saturday,103,182,232,67


In [16]:
# Number of Damaged cars in depot at the beginning of each day.
print("_________________________________________________________________________________")
print(f"Estimated number of damaged cars in depot at the beginning of each day: ")
print("_________________________________________________________________________________")

damaged_cars = pd.DataFrame(
    {
        "Day": [dayname[t] for t in days],
        "Glasgow": [round(nd['Glasgow',t].x) for t in days],
        "Manchester": [round(nd['Manchester',t].x) for t in days],
        "Birmingham": [round(nd['Birmingham',t].x) for t in days],
        "Plymouth": [round(nd['Plymouth',t].x) for t in days],
    }
)
damaged_cars.index=[''] * len(damaged_cars)
damaged_cars

_________________________________________________________________________________
Estimated number of damaged cars in depot at the beginning of each day: 
_________________________________________________________________________________


,Day,Glasgow,Manchester,Birmingham,Plymouth
,Monday,12,22,20,7
,Tuesday,11,24,20,8
,Wednesday,11,22,20,7
,Thursday,11,25,24,9
,Friday,19,22,20,11
,Saturday,17,22,20,13


In [17]:
# Undamaged car rented out from each depot and day.
print("_________________________________________________________________________________")
print(f"Estimated number of undamaged cars rented out from each depot and day: ")
print("_________________________________________________________________________________")

rentedOut = {}

for d in depots:
    for t in days:
        count = 0
        for d2 in depots:
            for r in rentDays:
                #print(f"Depot {d}, day {t}: cars rented out {tr[d,t].x}")
                count += pctUndamaged*pctFromToD[d,d2]*pctRent[r]*tr[d,t].x
        rentedOut[d,t] = round(count)
    

#print(rentedOut)

rentout_cars = pd.DataFrame(
    {
        "Day": [dayname[t] for t in days],
        "Glasgow": [round(rentedOut['Glasgow',t]) for t in days],
        "Manchester": [round(rentedOut['Manchester',t]) for t in days],
        "Birmingham": [round(rentedOut['Birmingham',t]) for t in days],
        "Plymouth": [round(rentedOut['Plymouth',t]) for t in days],
    }
)
rentout_cars.index=[''] * len(rentout_cars)
rentout_cars

_________________________________________________________________________________
Estimated number of undamaged cars rented out from each depot and day: 
_________________________________________________________________________________


,Day,Glasgow,Manchester,Birmingham,Plymouth
,Monday,85,204,86,59
,Tuesday,89,129,176,58
,Wednesday,90,72,218,50
,Thursday,75,203,100,72
,Friday,106,189,63,64
,Saturday,93,88,112,61


---
## References

H. Paul Williams, Model Building in Mathematical Programming, fifth edition.

Copyright © 2020 Gurobi Optimization, LLC